In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as image
import scipy.io
import copy

#from mpl_toolkits.mplot3d import Axes3D

from mpl_toolkits import mplot3d
from scipy.fft import fft, ifft, fft2, ifft2, fftshift, fftn, ifftn

In [ ]:
cwd = os.getcwd()
Kraken = scipy.io.loadmat('Kraken.mat')['Kraken']

In [ ]:
Kraken.shape

In [ ]:
L = 10
n = 64
threshold = 0.5

x2 = np.linspace(-L, L, n+1)
x = x2[:n]
y = x
z = x
k = (2*np.pi/(2*L))*np.concatenate([np.arange(n/2),-1 * np.flip(np.arange(n/2)+1)])

In [ ]:
X, Y, Z = np.meshgrid(x, y, z)
Kx, Ky, Kz = np.meshgrid(ks, ks, ks)

# for j in range(1, 50):
#     Un = np.reshape(Kraken[:, j-1], (n, n, n))
#     M = np.max(np.abs(Un))
#     V = abs(Un)/M
#     fig = go.Figure(data=go.Isosurface(
#     x=X.flatten(),
#     y=Y.flatten(),
#     z=Z.flatten(),
#     value=V.flatten(),
#     isomin = threshold,
#     isomax=1,
#     caps=dict(x_show=False, y_show=False)
#     ))
#     fig_widget = go.FigureWidget(fig)
#     #fig_widget.show()
#     #input("Press Enter to continue...")

In [ ]:
#realization 
Ut_sum = 0

for i in range(49):
    U = np.reshape(Kraken[:, i], (n, n, n),order = 'f')
    Ut = np.fft.fftn(U)
    Ut_sum += Ut

A1 = copy.copy(Ut_sum)

#Average

ave = Ut_sum/49
A2 = copy.copy(ave)

In [ ]:
max_x = np.unravel_index(np.argmax(np.abs(ave)), ave.shape)[0]
max_y = np.unravel_index(np.argmax(np.abs(ave)), ave.shape)[1]
max_z = np.unravel_index(np.argmax(np.abs(ave)), ave.shape)[2]

In [ ]:
print(max_x,max_y,max_z)

In [ ]:
A3 = copy.copy(k[max_x])
A4 = copy.copy(k[max_y])
A5 = copy.copy(k[max_z])

In [ ]:
##### define the filter

tau = 1/L

kx = k[max_x]
ky = k[max_y]
kz = k[max_z]

xx, yy, zz = np.meshgrid((k - kx),(k - ky),(k - kz), indexing='ij')
filter_g = np.exp(-tau*(xx**2+yy**2+zz**2))
A6 = copy.copy(filter_g)

In [ ]:
filter_g.shape

In [ ]:
loc_x = []
loc_y = []
loc_z = []
for i in range(49):
    U = np.reshape(Kraken[:, i], (n, n, n),order = 'f')
    Ut = np.fft.fftn(U)
    filter_U = Ut*filter_g
    unf = np.fft.ifftn(filter_U)
    loc_X = np.unravel_index(np.argmax(np.abs(unf)), unf.shape)[0]
    loc_Y = np.unravel_index(np.argmax(np.abs(unf)), unf.shape)[1]
    loc_Z = np.unravel_index(np.argmax(np.abs(unf)), unf.shape)[2]
    loc_x = np.append(loc_x,x[loc_X])
    loc_y = np.append(loc_y,x[loc_Y])
    loc_z = np.append(loc_z,x[loc_Z])

In [ ]:
A7 = loc_x.reshape((1,49))
A8 = loc_y.reshape((1,49))
A9 = loc_z.reshape((1,49))

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
import seaborn as sns
sns.set()


fig = plt.figure(figsize = ([10,10]))
param = np.linspace(0, 1, loc_x.size)
spl = make_interp_spline(param, np.c_[loc_x,loc_y], k=2) #(1)
xnew, y_smooth = spl(np.linspace(0, 1, x.size * 500)).T
plt.plot(xnew, y_smooth, linewidth= 2)
plt.title("Kraken's Path on (x,y) plane")
plt.xlabel("X-Axis")
plt.ylabel("Y-Axis")
plt.grid(True)
plt.savefig('path00.png',dpi=600)
plt.show()


In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
plt.title("Kraken's Path")

xx, yy = np.meshgrid(range(-8,8), range(-8,8))
zz = yy*0

arr = image.imread('pic1.png')
# 10 is equal length of x and y axises of your surface
stepX, stepY = 16. / arr.shape[0], 16. / arr.shape[1]

X1 = np.arange(-8, 8, stepX)
Y1 = np.arange(-8, 8, stepY)
X1, Y1 = np.meshgrid(X1, Y1)
ZZ = X1*0
# stride args allows to determine image quality 
# stride = 1 work slow

ax.scatter3D(loc_x, loc_y, loc_z, s=5, c=range(len(loc_x)), cmap='autumn', alpha = 0.5, label = "Location at each timestep")
ax.plot3D(loc_x, loc_y, 0, color = 'royalblue', linewidth=0.8,label="Kraken's path in 2D" )
ax.plot3D(loc_x, loc_y, loc_z, color = '#00A4CCFF', linewidth=1.5, label="Kraken's path in 3D")


ax.plot_surface(X1, Y1, ZZ+10, rstride=5, cstride=5, facecolors=arr, alpha = 0.1)
ax.plot_surface(xx, yy, zz, rstride=1, cstride=1, alpha = 0.1)


ax.legend(loc=1, prop={'size': 6})
ax.set_xlim3d(-8, 8)
ax.set_ylim3d(-8, 8)
ax.set_zlim3d(-10, 10)
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
ax.grid(False)
ax.view_init(20,120)
plt.show()